<a href="https://colab.research.google.com/github/eugenechi/mgmt467-analytics-portfolio/blob/main/Lab1_AI_Assisted_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 1: AI‑Assisted SQL Foundations (Colab)**
**Course:** MGMT 467 — AI‑Assisted Big Data Analytics in the Cloud  
**When:** Unit 1 · Week 3 (Thursday)  
**Goal:** Use *Gemini as a co‑pilot* to write and understand basic SQL against the **Superstore** dataset in **BigQuery**.  
**You will practice:** Prompt engineering → SQL generation → Query execution → Interpretation.

> ✅ **Deliverable (submit link in Brightspace):** A completed `Lab1_AI_Assisted_SQL.ipynb` with prompts, code, outputs.


## ✅ What you need before starting
- A Google account with access to **Google Colab**.
- A **Google Cloud Project** with **BigQuery** API enabled.
- A dataset named `superstore_data` and a table named `sales` (your instructor or TA will provide details).
- (Optional) A GitHub repo to store your work for your team.


## 0) Connect to class GitHub repo
## Install and import libraries
## Class Repo: bigDataNCloud/mgmt_467_resources

In [2]:
# Replace with your actual class repo URL
CLASS_REPO_URL = "https://github.com/bigDataNCloud/mgmt_467_resources.git"
CLASS_REPO_DIR = "/content/class-repo"

# If already cloned, skip re-downloading
import os
if not os.path.exists(CLASS_REPO_DIR):
    !git clone {CLASS_REPO_URL} {CLASS_REPO_DIR}
else:
    %cd {CLASS_REPO_DIR}
    !git pull

# Move into repo folder
%cd {CLASS_REPO_DIR}

print("✅ Class repository ready at", CLASS_REPO_DIR)

Cloning into '/content/class-repo'...
remote: Enumerating objects: 95, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 95 (delta 44), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (95/95), 66.24 KiB | 13.25 MiB/s, done.
Resolving deltas: 100% (44/44), done.
/content/class-repo
✅ Class repository ready at /content/class-repo


In [3]:
!pip -q install google-cloud-bigquery google-cloud-bigquery-storage db-dtypes pandas pyarrow kagglehub

In [4]:
from google.cloud import bigquery
from google.colab import auth
import pandas as pd
import os, json, textwrap, pathlib, pprint
print("Libraries imported.")

Libraries imported.


## 1) Authenticate to GCP and set your project
Run the cell below. When prompted, authorize Colab to access your Google Cloud resources.

In [5]:
auth.authenticate_user()
print("✅ Authenticated.")

# Set your GCP project (EDIT THIS!)
PROJECT_ID = "sapient-office-471119-g4"   # <-- EDIT ME
assert PROJECT_ID != "PROJECT_ID", "Please set PROJECT_ID to your GCP project id."
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID

# BigQuery client
bq = bigquery.Client(project=PROJECT_ID)
print("✅ BigQuery client created for:", PROJECT_ID)

✅ Authenticated.
✅ BigQuery client created for: sapient-office-471119-g4


### Quick connection test (optional)
This will try to read the first 5 rows from your Superstore table. If it fails, double‑check your dataset/table name.

In [13]:
DATASET = "superstore_data"
TABLE   = "sales"
BQ_TABLE = f"`{PROJECT_ID}.{DATASET}.{TABLE}`"

query = f"SELECT * FROM {BQ_TABLE} LIMIT 5"
try:
    df_preview = bq.query(query).to_dataframe()
    display(df_preview)
    print("✅ Connection OK.")
except Exception as e:
    print("⚠️ Could not query the table. Error below:")
    print(e)

⚠️ Could not query the table. Error below:
404 Not found: Table sapient-office-471119-g4:superstore_data.sales was not found in location US; reason: notFound, message: Not found: Table sapient-office-471119-g4:superstore_data.sales was not found in location US

Location: US
Job ID: f316d569-c0a9-4891-becf-73bb6cde7943



In [17]:
# Get the path to the downloaded file
download_path = "/root/.cache/kagglehub/datasets/anandaramg/global-superstore/versions/1/Global Superstore.txt"

# Read and print the first few lines of the file
try:
    with open(download_path, 'r', encoding='utf-8') as f:
        for i in range(10): # Read first 10 lines
            line = f.readline()
            print(line.strip())
except FileNotFoundError:
    print(f"File not found at: {download_path}")
except Exception as e:
    print(f"An error occurred while reading the file: {e}")

Category	City	Country	Customer ID	Customer Name	Discount	Market	记录数	Order Date	Order ID	Order Priority	Product ID	Product Name	Profit	Quantity	Region	Row ID	Sales	Segment	Ship Date	Ship Mode	Shipping Cost	State	Sub-Category	Year	Market2	weeknum
"Office Supplies"	"Los Angeles"	"United States"	"LS-172304"	"Lycoris Saunders"	0	"US"	1	2011-01-07 00:00:00.000	"CA-2011-130813"	"High"	"OFF-PA-10002005"	"Xerox 225"	9.3312	3	"West"	36624	19	"Consumer"	2011-01-09 00:00:00.000	"Second Class"	4.37	"California"	"Paper"	2011	"North America"	2
"Office Supplies"	"Los Angeles"	"United States"	"MV-174854"	"Mark Van Huff"	0	"US"	1	2011-01-21 00:00:00.000	"CA-2011-148614"	"Medium"	"OFF-PA-10002893"	"Wirebound Service Call Books, 5 1/2"" x 4"""	9.2928	2	"West"	37033	19	"Consumer"	2011-01-26 00:00:00.000	"Standard Class"	0.94	"California"	"Paper"	2011	"North America"	4
"Office Supplies"	"Los Angeles"	"United States"	"CS-121304"	"Chad Sievert"	0	"US"	1	2011-08-05 00:00:00.000	"CA-2011-118962"	"Medium"	"OFF-P

In [11]:
# List files in the KaggleHub download directory to verify the path
!ls /root/.cache/kagglehub/datasets/anandaramg/global-superstore/versions/1/

ls: cannot access '/root/.cache/kagglehub/datasets/anandaramg/global-superstore/versions/1/': No such file or directory


In [12]:
import kagglehub, os, pathlib

DATASET_REF = "anandaramg/global-superstore"   # you can change this to any public dataset reference

download_path = kagglehub.dataset_download(DATASET_REF)
print("Path to dataset files:", download_path)

# List files
p = pathlib.Path(download_path)
print("Files:")
for f in p.glob("**/*"):
    if f.is_file():
        print("-", f)

100%|██████████| 3.18M/3.18M [00:00<00:00, 138MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/anandaramg/global-superstore/versions/1
Files:
- /root/.cache/kagglehub/datasets/anandaramg/global-superstore/versions/1/Global Superstore.txt


In [8]:
# Make sure the BigQuery client is initialized
from google.cloud import bigquery
from google.colab import auth
import os

auth.authenticate_user()
print("✅ Authenticated.")

# Set your GCP project (EDIT THIS!)
PROJECT_ID = "sapient-office-471119-g4"   # <-- EDIT ME
assert PROJECT_ID != "PROJECT_ID", "Please set PROJECT_ID to your GCP project id."
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID

# BigQuery client
bq = bigquery.Client(project=PROJECT_ID)
print("✅ BigQuery client created for:", PROJECT_ID)

✅ Authenticated.
✅ BigQuery client created for: sapient-office-471119-g4


In [20]:
# Define dataset and table IDs
DATASET_ID = "superstore_data" # Replace with your desired dataset ID
TABLE_ID = "sales" # Replace with your desired table ID
PROJECT_ID = "sapient-office-471119-g4" # Replace with your project ID

# Construct BigQuery client
client = bigquery.Client(project=PROJECT_ID)

# Create dataset if it doesn't exist
dataset_ref = client.dataset(DATASET_ID)
try:
    client.get_dataset(dataset_ref)
    print(f"Dataset {DATASET_ID} already exists.")
except Exception as e:
    print(f"Dataset {DATASET_ID} not found. Creating dataset.")
    dataset = bigquery.Dataset(dataset_ref)
    dataset.location = "US" # Set the location to US
    client.create_dataset(dataset)
    print(f"Dataset {DATASET_ID} created.")

# Define table reference
table_ref = dataset_ref.table(TABLE_ID)

# Define the schema for the table
schema = [
    bigquery.SchemaField("Category", "STRING"), # Added from file inspection
    bigquery.SchemaField("City", "STRING"), # Added from file inspection
    bigquery.SchemaField("Country", "STRING"), # Added from file inspection
    bigquery.SchemaField("Customer_ID", "STRING"),
    bigquery.SchemaField("Customer_Name", "STRING"),
    bigquery.SchemaField("Discount", "BIGNUMERIC"), # Use BIGNUMERIC for decimal discount values
    bigquery.SchemaField("Market", "STRING"),
    bigquery.SchemaField("Record_Count", "INT64"), # Added based on "记录数" column in file
    bigquery.SchemaField("Order_Date", "STRING"), # Can be converted to DATE later if needed
    bigquery.SchemaField("Order_ID", "STRING"),
    bigquery.SchemaField("Order_Priority", "STRING"),
    bigquery.SchemaField("Product_ID", "STRING"), # Added from file inspection
    bigquery.SchemaField("Product_Name", "STRING"),
    bigquery.SchemaField("Profit", "BIGNUMERIC"),   # Use BIGNUMERIC for currency/profit data
    bigquery.SchemaField("Quantity", "INT64"),
    bigquery.SchemaField("Region", "STRING"),
    bigquery.SchemaField("Row_ID", "INT64"),
    bigquery.SchemaField("Sales", "BIGNUMERIC"), # Use BIGNUMERIC for currency/sales data
    bigquery.SchemaField("Segment", "STRING"),
    bigquery.SchemaField("Ship_Date", "STRING"),  # Can be converted to DATE later if needed
    bigquery.SchemaField("Ship_Mode", "STRING"),
    bigquery.SchemaField("Shipping_Cost", "BIGNUMERIC"), # Use BIGNUMERIC for currency data
    bigquery.SchemaField("State", "STRING"), # Added from file inspection
    bigquery.SchemaField("Sub_Category", "STRING"),
    bigquery.SchemaField("Year", "INT64"), # Added based on "Year" column in file
    bigquery.SchemaField("Market2_weeknum", "STRING"), # Changed to STRING based on error message
    bigquery.SchemaField("Postal_Code", "STRING"), # Keep as STRING due to potential leading zeros - Reordered to match file
]


# Configure the job to load data
job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,
    schema=schema,  # Use the defined schema
    skip_leading_rows=1, # Skip header row
    field_delimiter='\t', # Specify tab as the delimiter
    quote_character='"', # Specify double quote as the quote character
)

# Get the path to the downloaded file
download_path = "/root/.cache/kagglehub/datasets/anandaramg/global-superstore/versions/1/Global Superstore.txt"

# Load data from the file into the table
with open(download_path, "rb") as source_file:
    job = client.load_table_from_file(
        source_file, table_ref, job_config=job_config
    )

print("Loading data. Please wait...")
job.result()  # Waits for the job to complete.

print(f"Data loaded into {PROJECT_ID}.{DATASET_ID}.{TABLE_ID}")

Dataset superstore_data already exists.
Loading data. Please wait...
Data loaded into sapient-office-471119-g4.superstore_data.sales


## 2) (Optional) Download a dataset with **KaggleHub**
Use **KaggleHub** to pull sample data locally into Colab (e.g., for offline exploration or to stage for GCS).
> This is **optional** for Lab 1 (we focus on BigQuery), but useful to practice data access workflows.
> https://www.kaggle.com/datasets/anandaramg/global-superstore

In [21]:
import kagglehub, os, pathlib

DATASET_REF = "anandaramg/global-superstore"   # you can change this to any public dataset reference

download_path = kagglehub.dataset_download(DATASET_REF)
print("Path to dataset files:", download_path)

# List files
p = pathlib.Path(download_path)
print("Files:")
for f in p.glob("**/*"):
    if f.is_file():
        print("-", f)


Using Colab cache for faster access to the 'global-superstore' dataset.
Path to dataset files: /kaggle/input/global-superstore
Files:
- /kaggle/input/global-superstore/Global Superstore.txt


## 3) Prompting approach for Lab 1
You'll **paste prompts into Gemini** (in a separate tab), get back **SQL**, then paste that SQL here to execute.  
We’ll practice three core question types:
- **“What”** (lists & filters) → `SELECT`, `WHERE`, `DISTINCT`
- **“How many”** (counts by category) → `COUNT`, `GROUP BY`
- **“Who is best”** (rank/limit by metric) → `SUM`, `ORDER BY`, `LIMIT`

### 3A) “What” Question — `SELECT`, `WHERE`, `DISTINCT`
**Business Question:** *A manager wants a list of all **unique** product sub‑categories sold in the **West** region.*

**Paste this prompt into Gemini (no edits needed except the project id if you used a different table path):**
```
# TASK: Generate a BigQuery SQL query.
# CONTEXT: The table is `[YOUR_PROJECT_ID].superstore_data.sales`.
# GOAL: Find all the unique values in the 'Sub_Category' column, but only for rows where the 'Region' column is exactly 'West'.
```

**Then paste the SQL you get into the cell below** (replace the placeholder) and run it.

In [22]:
sql_str = """
SELECT DISTINCT Sub_Category
FROM `YOUR_PROJECT_ID_HERE.superstore_data.sales`
WHERE Region = 'West'
ORDER BY Sub_Category
"""
sql_str = sql_str.replace("YOUR_PROJECT_ID_HERE", PROJECT_ID)

df_what = bq.query(sql_str).to_dataframe()
print(f"Rows: {len(df_what)}")
display(df_what.head(20))

Rows: 17


,Sub_Category
0,Accessories
1,Appliances
2,Art
3,Binders
4,Bookcases
5,Chairs
6,Copiers
7,Envelopes
8,Fasteners
9,Furnishings


No charts were generated by quickchart


In [ ]:
sql_str = """
-- PASTE the SQL from Gemini here
SELECT DISTINCT Sub_Category
FROM `YOUR_PROJECT_ID_HERE.superstore_data.sales`
WHERE Region = 'West'
ORDER BY Sub_Category
"""
sql_str = sql_str.replace("YOUR_PROJECT_ID_HERE", PROJECT_ID)

df_what = bq.query(sql_str).to_dataframe()
print(f"Rows: {len(df_what)}")
display(df_what.head(20))

In [23]:
sql_str = """
SELECT DISTINCT Sub_Category
FROM `YOUR_PROJECT_ID_HERE.superstore_data.sales`
WHERE Region = 'West'
ORDER BY Sub_Category
"""
sql_str = sql_str.replace("YOUR_PROJECT_ID_HERE", PROJECT_ID)

df_what = bq.query(sql_str).to_dataframe()
print(f"Rows: {len(df_what)}")
display(df_what.head(20))

Rows: 17


,Sub_Category
0,Accessories
1,Appliances
2,Art
3,Binders
4,Bookcases
5,Chairs
6,Copiers
7,Envelopes
8,Fasteners
9,Furnishings


**Explain it back (metacognition):** In Gemini, paste:  
> *Explain the following SQL query line by line: [paste your SQL]*  
Summarize what you learned here:

> **Notes:**

SELECT DISTINCT Sub_Category
FROM `YOUR_PROJECT_ID_HERE.superstore_data.sales`
WHERE Region = 'West'
ORDER BY Sub_Category

### 3B) “How many” Question — `COUNT`, `GROUP BY`
**Business Question:** *How many orders were placed in each **Ship Mode**?*

**Gemini prompt:**
```
# TASK: Generate a BigQuery SQL query.
# CONTEXT: The table is `[YOUR_PROJECT_ID].superstore_data.sales`.
# GOAL: Count the total number of records for each unique value in the 'Ship_Mode' column.
# The final result should have two columns: 'Ship_Mode' and 'order_count'.
```


In [24]:
sql_str = """
SELECT Ship_Mode, COUNT(*) AS order_count
FROM `YOUR_PROJECT_ID_HERE.superstore_data.sales`
GROUP BY Ship_Mode
ORDER BY order_count DESC
"""
sql_str = sql_str.replace("YOUR_PROJECT_ID_HERE", PROJECT_ID)

df_howmany = bq.query(sql_str).to_dataframe()
display(df_howmany)

,Ship_Mode,order_count
0,Standard Class,30775
1,Second Class,10309
2,First Class,7505
3,Same Day,2701


In [25]:
sql_str = """
-- PASTE the SQL from Gemini here
SELECT Ship_Mode, COUNT(*) AS order_count
FROM `YOUR_PROJECT_ID_HERE.superstore_data.sales`
GROUP BY Ship_Mode
ORDER BY order_count DESC
"""
sql_str = sql_str.replace("YOUR_PROJECT_ID_HERE", PROJECT_ID)

df_howmany = bq.query(sql_str).to_dataframe()
display(df_howmany)

,Ship_Mode,order_count
0,Standard Class,30775
1,Second Class,10309
2,First Class,7505
3,Same Day,2701


### 3C) “Who is best” Question — `SUM`, `ORDER BY`, `LIMIT`
**Business Question:** *Identify the **top 5 most profitable customers**.*

**Gemini prompt:**
```
# TASK: Generate a BigQuery SQL query.
# CONTEXT: The table is `[YOUR_PROJECT_ID].superstore_data.sales`. The customer identifier is 'Customer_ID'.
# GOAL: Calculate the sum of 'Profit' for each customer. The final output should show the 'Customer_ID' and their total profit, sorted from highest to lowest profit, and limited to only the top 5 results.
```


In [26]:
sql_str = """
SELECT Customer_ID, SUM(Profit) AS total_profit
FROM `YOUR_PROJECT_ID_HERE.superstore_data.sales`
GROUP BY Customer_ID
ORDER BY total_profit DESC
LIMIT 5
"""
sql_str = sql_str.replace("YOUR_PROJECT_ID_HERE", PROJECT_ID)

df_best = bq.query(sql_str).to_dataframe()
display(df_best)

,Customer_ID,total_profit
0,TC-209804,8981.32390000000000000000000000000000000000
1,RB-193604,6976.09590000000000000000000000000000000000
2,SC-200954,5757.41190000000000000000000000000000000000
3,HL-150404,5622.42920000000000000000000000000000000000
4,AB-101054,5444.80550000000000000000000000000000000000


In [27]:
sql_str = """
-- PASTE the SQL from Gemini here
SELECT Customer_ID, SUM(Profit) AS total_profit
FROM `YOUR_PROJECT_ID_HERE.superstore_data.sales`
GROUP BY Customer_ID
ORDER BY total_profit DESC
LIMIT 5
"""
sql_str = sql_str.replace("YOUR_PROJECT_ID_HERE", PROJECT_ID)

df_best = bq.query(sql_str).to_dataframe()
display(df_best)

,Customer_ID,total_profit
0,TC-209804,8981.32390000000000000000000000000000000000
1,RB-193604,6976.09590000000000000000000000000000000000
2,SC-200954,5757.41190000000000000000000000000000000000
3,HL-150404,5622.42920000000000000000000000000000000000
4,AB-101054,5444.80550000000000000000000000000000000000


## 4) Challenge prompts (author your own)
Write **your own precise prompt** in a text cell (or comment) for each question below, then get SQL from Gemini and run it in the provided code cells.

**Challenge 1:** *What is the **average discount** for products in the **Technology** category sold in the **East** region?*  
**Challenge 2:** *How many **unique customers** has each **Segment** (Consumer/Corporate/Home Office) served?*

In [28]:
sql_str = """
-- PASTE your Gemini-generated SQL here
SELECT AVG(Discount) AS avg_discount
FROM `YOUR_PROJECT_ID_HERE.superstore_data.sales`
WHERE Category = 'Technology' AND Region = 'East'
"""
sql_str = sql_str.replace("YOUR_PROJECT_ID_HERE", PROJECT_ID)
df_ch1 = bq.query(sql_str).to_dataframe()
display(df_ch1)

,avg_discount
0,0.14336448598130841121495327102803738318


In [29]:
sql_str = """
-- PASTE your Gemini-generated SQL here
SELECT Segment, COUNT(DISTINCT Customer_ID) AS unique_customers
FROM `YOUR_PROJECT_ID_HERE.superstore_data.sales`
GROUP BY Segment
ORDER BY unique_customers DESC
"""
sql_str = sql_str.replace("YOUR_PROJECT_ID_HERE", PROJECT_ID)
df_ch2 = bq.query(sql_str).to_dataframe()
display(df_ch2)

,Segment,unique_customers
0,Consumer,2509
1,Corporate,1457
2,Home Office,907


## 5) Reflection (DIVE mindset)
- **Discover:** What did you find first?  
- **Investigate:** What alternate query or filter changed the story?  
- **Validate:** Where could the AI‑generated SQL be wrong or incomplete? How did you check?  
- **Extend:** Which stakeholder could use your results tomorrow? What action should they take?

- I found that connecting and setting up my GCP along with the data sets was kind of complicated until I queried Gemini to help me with things that confused me such as "bucketes". No specific alternate query changed it, but I did keep asking Gemini to speficially help me with importing the kaggle dataset to my GCP. The first few times I asked it to help out with it, Gemini gave me an answer that did not work as it was telling me to use a certain file path that didn't make sense on my computer. I simply recognized that it wasn't correct and told Gemini, in which it swiftly fixed its mistake. In reality, the Customer Service Team will benefit a lot from unique_customers majority being consumers.

In [30]:
display(df_ch2)

,Segment,unique_customers
0,Consumer,2509
1,Corporate,1457
2,Home Office,907


## 6) Save your work to GitHub (pick one of the options)
**Option A (recommended):** In Colab, go to **File → Save a copy in GitHub…** and select your team repo + folder (e.g., `labs/Unit1/`).  
**Option B (CLI, if you know git):**
```bash
# (In Colab) mount Drive, then clone/pull/push as usual with a PAT
# Be careful to NOT store secrets in the notebook.
```
Name the file **`Lab1_AI_Assisted_SQL.ipynb`** and push it to your team repo.